<a href="https://colab.research.google.com/github/mratanusarkar/Web-Scraping-tickertapeIN/blob/feature%2Ffilter-and-sort/Notebooks/filter_and_sort_tickertapeIN_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filtering and Sorting data scraped from tickertape.in

Input: 
* scraped JSON data (ETFs or Stocks)
* list of keys and values and comparison type for filtering
* list of keys and the order of the values for sorting

Output: JSON data with the filters & sortings applied on the input data

## Import Packages

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import json
import time
import datetime as dt

import matplotlib.pyplot as plt
import plotly.express as px

## Input JSON Data from Scraping Scripts/Notebooks

In [2]:
with open('all-stocks-tickertape-data.json', 'r') as fp:
    stocksData = json.load(fp)

stocksData[0]

{'forecasts': {'buyRecommendation': None, 'forecast': None},
 'investmentChecklist': {'dividendReturns': 'negative',
  'entryPoint': 'positive',
  'intrinsicValue': 'neutral',
  'noRedFlags': 'positive',
  'roeVsFdRates': 'negative'},
 'keyMetrics': {'DividendYield': None,
  'PBRatio': '0.13',
  'PERatio': '-4.05',
  'SectorDivYld': '0.65%',
  'SectorPB': '5.21',
  'SectorPE': '68.27'},
 'marketcap': 'Smallcap',
 'name': 'A & M Febcon Ltd',
 'overview': {'absoluteChange': '0.74%',
  'capDesc': 'With a market cap of ₹1 cr, stock is ranked 4,213',
  'capType': 'Smallcap',
  'currentPrice': '1.35',
  'percentageChange': '-0.01',
  'riskDesc': 'Stock is 2.78x as volatile as Nifty',
  'riskType': 'Moderate Risk',
  'sectorDesc': 'Industrial Machinery',
  'sectorType': 'Industrials'},
 'price': '1.35',
 'profile': 'Company Profile: A & M Febcon is engaged in the services of engineering which is applied to the planning, designing and control of industrial operations and in the business of Ind

In [3]:
with open('all-etfs-tickertape-data.json', 'r') as fp:
    etfsData = json.load(fp)

etfsData[0]

{'investmentChecklist': {'entryPoint': 'negative',
  'expenseRatio': 'negative',
  'nav': 'neutral',
  'returnVsFdRates': 'positive',
  'trackingError': 'neutral'},
 'keyMetrics': {'AUM': '₹ 347.68cr',
  'ExpenseRatio': '0.58%',
  'RealtimeNAV': None,
  'SectorExpenseRatio': '0.61%',
  'SectorTrackingError': None,
  'TrackingError': None},
 'marketcap': 'Gold',
 'name': 'Aditya BSL Gold ETF',
 'overview': {'absoluteChange': '0.41%',
  'capDesc': 'ETF tracks gold prices',
  'capType': 'Gold',
  'currentPrice': '48.58',
  'percentageChange': '+0.20',
  'riskDesc': 'Average daily traded value of the ETF  is medium',
  'riskType': 'Medium Liquidity',
  'sectorDesc': 'Gold',
  'sectorType': 'ETF'},
 'price': '48.58',
 'profile': "AMC profile: Aditya Birla Sun Life AMC Limited, is a joint venture between the Aditya Birla Group and the Sun Life Financial Inc. of Canada. The joint venture brings together Aditya Birla Group's experience in the Indian market and Sun Life's global experience.",
 

## Input Filter/Sort Parameters

In [4]:
# TODO

In [25]:
# # testing a sample input
# inp = input("enter a value:")
# print("recieved input:", inp)
# print(type(inp))
# print(inp == "")

enter a value:1
recieved input: 1
<class 'str'>
False


## Processing the Data

#### 1. Stock Data

In [5]:
# handpicking the important filter-sort data fields

index = 0
print('name:', stocksData[index]['name'])
print('marcap:', stocksData[index]['marketcap'])
print('sector:', stocksData[index]['sector'])
print('risk:', stocksData[index]['risk'])
print('price:', stocksData[index]['price'])
print('intr_val:', stocksData[index]['investmentChecklist']['intrinsicValue'])
print('roe_fd:', stocksData[index]['investmentChecklist']['roeVsFdRates'])
print('div_ret:', stocksData[index]['investmentChecklist']['dividendReturns'])
print('entry_pt:', stocksData[index]['investmentChecklist']['entryPoint'])
print('red_flag:', stocksData[index]['investmentChecklist']['noRedFlags'])
print('recommend:', stocksData[index]['forecasts']['buyRecommendation'])

name: A & M Febcon Ltd
marcap: Smallcap
sector: Industrials
risk: Moderate Risk
price: 1.35
intr_val: neutral
roe_fd: negative
div_ret: negative
entry_pt: positive
red_flag: positive
recommend: None


In [6]:
# filter out the required parameters from the full data

selectedDataList = []
for i, data in enumerate(stocksData):
    if len(data) == 0:
        continue
    selectedData = {}
    selectedData['name'] = data['name']
    selectedData['marcap'] = data['marketcap']
    selectedData['sector'] = data['sector']
    selectedData['risk'] = data['risk']
    selectedData['price'] = data['price']
    selectedData['intr_val'] = data['investmentChecklist']['intrinsicValue']
    selectedData['roe_fd'] = data['investmentChecklist']['roeVsFdRates']
    selectedData['div_ret'] = data['investmentChecklist']['dividendReturns']
    selectedData['entry_pt'] = data['investmentChecklist']['entryPoint']
    selectedData['red_flag'] = data['investmentChecklist']['noRedFlags'] if 'noRedFlags' in data['investmentChecklist'] else 'negative'
    selectedData['recommend'] = data['forecasts']['buyRecommendation']

    selectedDataList.append(selectedData)

In [7]:
print(len(stocksData))
print(len(selectedDataList))

4487
4404


In [8]:
# convert into pandas dataframe for easier processing

stockDf = pd.json_normalize(selectedDataList)
stockDf

,name,marcap,sector,risk,price,intr_val,roe_fd,div_ret,entry_pt,red_flag,recommend
0,A & M Febcon Ltd,Smallcap,Industrials,Moderate Risk,1.35,neutral,negative,negative,positive,positive,None
1,A B Cotspin India Ltd,Smallcap,Consumer Discretionary,High Risk,44.90,positive,neutral,neutral,positive,positive,None
2,A B Infrabuild Ltd,Smallcap,Industrials,High Risk,11.10,neutral,positive,neutral,negative,positive,None
3,A F Enterprises Ltd,Smallcap,Financials,Moderate Risk,135.45,neutral,positive,negative,negative,negative,None
4,A Infrastructure Ltd,Smallcap,Industrials,High Risk,64.15,neutral,negative,negative,positive,negative,None
...,...,...,...,...,...,...,...,...,...,...,...
4399,52 Weeks Entertainment Ltd,Smallcap,Communication Services,High Risk,2.77,neutral,negative,negative,positive,negative,None
4400,5Paisa Capital Ltd,Smallcap,Financials,High Risk,364.80,negative,negative,negative,positive,positive,100%
4401,63 Moons Technologies Ltd,Smallcap,Information Technology,High Risk,294.35,neutral,negative,positive,positive,negative,None
4402,7NR Retail Ltd,Smallcap,,High Risk,2.25,neutral,negative,negative,positive,negative,None


In [9]:
stockDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       4404 non-null   object
 1   marcap     4404 non-null   object
 2   sector     4404 non-null   object
 3   risk       4404 non-null   object
 4   price      4404 non-null   object
 5   intr_val   4404 non-null   object
 6   roe_fd     4404 non-null   object
 7   div_ret    4404 non-null   object
 8   entry_pt   4404 non-null   object
 9   red_flag   4404 non-null   object
 10  recommend  696 non-null    object
dtypes: object(11)
memory usage: 378.6+ KB


In [10]:
# categorical columns

cat_cols = ['marcap', 'sector', 'risk', 'intr_val', 'roe_fd', 'div_ret', 'entry_pt', 'red_flag', 'recommend']
for col in cat_cols:
    print(col + ': ', stockDf[col].unique())

marcap:  ['Smallcap' 'Unknown' 'Midcap' 'Largecap']
sector:  ['Industrials' 'Consumer Discretionary' 'Financials' 'Materials' ''
 'Information Technology' 'Consumer Staples' 'Real Estate' 'Health Care'
 'Energy' 'Communication Services' 'Utilities' 'Unknown']
risk:  ['Moderate Risk' 'High Risk' 'Low Risk']
intr_val:  ['neutral' 'positive' 'negative']
roe_fd:  ['negative' 'neutral' 'positive']
div_ret:  ['negative' 'neutral' 'positive']
entry_pt:  ['positive' 'negative' 'neutral']
red_flag:  ['positive' 'negative']
recommend:  [None '42%' '85%' '62%' '92%' '100%' '60%' '40%' '71%' '0%' '95%' '33%'
 '83%' '89%' '50%' '70%' '44%' '53%' '63%' '80%' '79%' '94%' '46%' '22%'
 '21%' '88%' '67%' '76%' '75%' '73%' '61%' '65%' '82%' '25%' '16%' '81%'
 '97%' '93%' '55%' '90%' '48%' '77%' '74%' '43%' '78%' '87%' '72%' '58%'
 '37%' '91%' '86%' '35%' '17%' '64%' '84%' '11%' '54%' '59%' '49%' '52%'
 '20%' '98%' '68%' '13%' '69%' '45%' '10%' '36%' '31%' '96%' '39%' '30%'
 '5%' '38%' '57%' '14%' '28%' '

In [11]:
stockDf["marcap"] = stockDf["marcap"].replace('Unknown', '0').replace('Smallcap', '1').replace('Midcap', '2').replace('Largecap', '3').astype(int)
stockDf["risk"] = stockDf["risk"].replace('Unknown', '0').replace('Low Risk', '1').replace('Moderate Risk', '2').replace('High Risk', '3').astype(int)
stockDf["intr_val"] = stockDf["intr_val"].replace('negative', '-1').replace('neutral', '0').replace('positive', '1').astype(int)
stockDf["roe_fd"] = stockDf["roe_fd"].replace('negative', '-1').replace('neutral', '0').replace('positive', '1').astype(int)
stockDf["div_ret"] = stockDf["div_ret"].replace('negative', '-1').replace('neutral', '0').replace('positive', '1').astype(int)
stockDf["entry_pt"] = stockDf["entry_pt"].replace('negative', '-1').replace('neutral', '0').replace('positive', '1').astype(int)
stockDf["red_flag"] = stockDf["intr_val"].replace('negative', '-1').replace('neutral', '0').replace('positive', '1').astype(int)

In [12]:
# decimal column

stockDf["recommend"].fillna(value=np.nan).replace('[\%]', '', regex=True).astype(float)

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
4399      NaN
4400    100.0
4401      NaN
4402      NaN
4403      NaN
Name: recommend, Length: 4404, dtype: float64

In [13]:
stockDf["recommend"] = stockDf["recommend"].fillna(value=np.nan).replace('[\%]', '', regex=True).astype(float)

In [14]:
# pricing column

stockDf["price"].replace('[\—,]', '', regex=True).replace('', 'nan').astype(float)

0         1.35
1        44.90
2        11.10
3       135.45
4        64.15
         ...  
4399      2.77
4400    364.80
4401    294.35
4402      2.25
4403     22.40
Name: price, Length: 4404, dtype: float64

In [15]:
stockDf["price"] = stockDf["price"].replace('[\—,]', '', regex=True).replace('', 'nan').astype(float)

In [16]:
# final dataframe
stockDf

,name,marcap,sector,risk,price,intr_val,roe_fd,div_ret,entry_pt,red_flag,recommend
0,A & M Febcon Ltd,1,Industrials,2,1.35,0,-1,-1,1,0,NaN
1,A B Cotspin India Ltd,1,Consumer Discretionary,3,44.90,1,0,0,1,1,NaN
2,A B Infrabuild Ltd,1,Industrials,3,11.10,0,1,0,-1,0,NaN
3,A F Enterprises Ltd,1,Financials,2,135.45,0,1,-1,-1,0,NaN
4,A Infrastructure Ltd,1,Industrials,3,64.15,0,-1,-1,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4399,52 Weeks Entertainment Ltd,1,Communication Services,3,2.77,0,-1,-1,1,0,NaN
4400,5Paisa Capital Ltd,1,Financials,3,364.80,-1,-1,-1,1,-1,100.0
4401,63 Moons Technologies Ltd,1,Information Technology,3,294.35,0,-1,1,1,0,NaN
4402,7NR Retail Ltd,1,,3,2.25,0,-1,-1,1,0,NaN


In [17]:
stockDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       4404 non-null   object 
 1   marcap     4404 non-null   int64  
 2   sector     4404 non-null   object 
 3   risk       4404 non-null   int64  
 4   price      4398 non-null   float64
 5   intr_val   4404 non-null   int64  
 6   roe_fd     4404 non-null   int64  
 7   div_ret    4404 non-null   int64  
 8   entry_pt   4404 non-null   int64  
 9   red_flag   4404 non-null   int64  
 10  recommend  696 non-null    float64
dtypes: float64(2), int64(7), object(2)
memory usage: 378.6+ KB


#### 2. ETF Data

In [18]:
# TODO

## Data Analysis and EDA

In [19]:
# TODO

## Filter and Sort Data based on Input Parameters

In [ ]:
# Filter

In [26]:
stockDf.head()

,name,marcap,sector,risk,price,intr_val,roe_fd,div_ret,entry_pt,red_flag,recommend
0,A & M Febcon Ltd,1,Industrials,2,1.35,0,-1,-1,1,0,NaN
1,A B Cotspin India Ltd,1,Consumer Discretionary,3,44.90,1,0,0,1,1,NaN
2,A B Infrabuild Ltd,1,Industrials,3,11.10,0,1,0,-1,0,NaN
3,A F Enterprises Ltd,1,Financials,2,135.45,0,1,-1,-1,0,NaN
4,A Infrastructure Ltd,1,Industrials,3,64.15,0,-1,-1,1,0,NaN


In [38]:
stockDf.loc[stockDf['intr_val'] > 0].loc[stockDf['roe_fd'] > 0].loc[stockDf['div_ret'] > 0].loc[stockDf['entry_pt'] > 0].loc[stockDf['red_flag'] > 0].loc[stockDf.recommend.notna()]

,name,marcap,sector,risk,price,intr_val,roe_fd,div_ret,entry_pt,red_flag,recommend
400,Avadh Sugar & Energy Ltd,1,Consumer Staples,3,833.75,1,1,1,1,1,100.0
454,Bajaj Auto Ltd,3,Consumer Discretionary,1,3714.60,1,1,1,1,1,76.0
455,Bajaj Consumer Care Ltd,1,Consumer Staples,1,172.85,1,1,1,1,1,75.0
461,Bajaj Holdings and Investment Ltd,3,Financials,1,5204.40,1,1,1,1,1,100.0
542,Bharat Petroleum Corporation Ltd,3,Energy,1,371.75,1,1,1,1,1,97.0
...,...,...,...,...,...,...,...,...,...,...,...
4005,Torrent Power Ltd,2,Utilities,2,570.50,1,1,1,1,1,22.0
4043,Triveni Engineering and Industries Ltd,1,Consumer Staples,3,339.05,1,1,1,1,1,100.0
4261,Visaka Industries Ltd,1,Materials,2,616.40,1,1,1,1,1,100.0
4313,Welspun India Ltd,1,Consumer Discretionary,3,94.30,1,1,1,1,1,83.0


In [63]:
filter = pd.Series([True] * stockDf.shape[0])

filter = filter & (stockDf['intr_val'] > 0)
filter = filter & (stockDf['roe_fd'] > 0)
filter = filter & (stockDf['div_ret'] > 0)
filter = filter & (stockDf['entry_pt'] > 0)
filter = filter & (stockDf['red_flag'] > 0)

filter = filter & (stockDf['recommend'].notna())
filter = filter & (stockDf['price'].notna())

filter

0       False
1       False
2       False
3       False
4       False
        ...  
4399    False
4400    False
4401    False
4402    False
4403    False
Length: 4404, dtype: bool

In [65]:
resultDf = stockDf.loc[filter, :]
resultDf

,name,marcap,sector,risk,price,intr_val,roe_fd,div_ret,entry_pt,red_flag,recommend
400,Avadh Sugar & Energy Ltd,1,Consumer Staples,3,833.75,1,1,1,1,1,100.0
454,Bajaj Auto Ltd,3,Consumer Discretionary,1,3714.60,1,1,1,1,1,76.0
455,Bajaj Consumer Care Ltd,1,Consumer Staples,1,172.85,1,1,1,1,1,75.0
461,Bajaj Holdings and Investment Ltd,3,Financials,1,5204.40,1,1,1,1,1,100.0
542,Bharat Petroleum Corporation Ltd,3,Energy,1,371.75,1,1,1,1,1,97.0
...,...,...,...,...,...,...,...,...,...,...,...
4005,Torrent Power Ltd,2,Utilities,2,570.50,1,1,1,1,1,22.0
4043,Triveni Engineering and Industries Ltd,1,Consumer Staples,3,339.05,1,1,1,1,1,100.0
4261,Visaka Industries Ltd,1,Materials,2,616.40,1,1,1,1,1,100.0
4313,Welspun India Ltd,1,Consumer Discretionary,3,94.30,1,1,1,1,1,83.0


In [57]:
# Sort and Order

In [67]:
resultDf = resultDf.sort_values(by='price', ascending=True).sort_values(by='recommend', ascending=False)
resultDf

,name,marcap,sector,risk,price,intr_val,roe_fd,div_ret,entry_pt,red_flag,recommend
3254,SJVN Ltd,1,Utilities,1,28.95,1,1,1,1,1,100.0
2441,Moil Ltd,1,Materials,2,182.40,1,1,1,1,1,100.0
2827,Patels Airtemp (India) Ltd,1,Industrials,3,262.00,1,1,1,1,1,100.0
2629,Nitin Spinners Ltd,1,Consumer Discretionary,3,268.05,1,1,1,1,1,100.0
1273,Garden Reach Shipbuilders & Engineers Ltd,1,Industrials,3,304.65,1,1,1,1,1,100.0
...,...,...,...,...,...,...,...,...,...,...,...
1546,Hindustan Zinc Ltd,3,Materials,2,344.05,1,1,1,1,1,20.0
3980,Time Technoplast Ltd,1,Materials,2,88.00,1,1,1,1,1,0.0
3885,TVS Srichakra Ltd,1,Consumer Discretionary,1,1694.95,1,1,1,1,1,0.0
1218,GHCL Ltd,1,Materials,3,592.10,1,1,1,1,1,0.0


In [ ]:
######################### ANALYSING THE RESULTS ON MARKET CAP BASIS #########################

In [89]:
# Small Cap
resultDf.loc[resultDf.marcap == 1, ['name', 'sector', 'price', 'recommend']].sort_values(by='price', ascending=True).sort_values(by='recommend', ascending=False)

,name,sector,price,recommend
3254,SJVN Ltd,Utilities,28.95,100.0
634,CARE Ratings Ltd,Financials,554.95,100.0
1273,Garden Reach Shipbuilders & Engineers Ltd,Industrials,304.65,100.0
4043,Triveni Engineering and Industries Ltd,Consumer Staples,339.05,100.0
3055,Rail Vikas Nigam Ltd,Financials,34.35,100.0
1790,JK Paper Ltd,Materials,365.10,100.0
2646,Nucleus Software Exports Ltd,Information Technology,440.60,100.0
1383,Graphite India Ltd,Industrials,530.70,100.0
891,Dalmia Bharat Sugar and Industries Ltd,Consumer Staples,539.45,100.0
4261,Visaka Industries Ltd,Materials,616.40,100.0


In [90]:
# Mid Cap
resultDf.loc[resultDf.marcap == 2, ['name', 'sector', 'price', 'recommend']].sort_values(by='price', ascending=True).sort_values(by='recommend', ascending=False)

,name,sector,price,recommend
2902,Power Finance Corporation Ltd,Financials,119.00,100.0
3020,REC Limited,Financials,130.80,100.0
817,Coromandel International Ltd,Materials,856.00,100.0
2700,Oracle Financial Services Software Ltd,Information Technology,3587.15,100.0
1543,Hindustan Petroleum Corp Ltd,Energy,283.35,82.0
1578,ICICI Securities Ltd,Financials,627.20,80.0
2547,National Aluminium Co Ltd,Materials,119.70,78.0
2513,NMDC Ltd,Materials,168.25,75.0
2106,LIC Housing Finance Ltd,Financials,376.75,70.0
2669,Oil India Ltd,Energy,235.85,68.0


In [91]:
# Large Cap
resultDf.loc[resultDf.marcap == 3, ['name', 'sector', 'price', 'recommend']].sort_values(by='price', ascending=True).sort_values(by='recommend', ascending=False)

,name,sector,price,recommend
461,Bajaj Holdings and Investment Ltd,Financials,5204.40,100.0
542,Bharat Petroleum Corporation Ltd,Energy,371.75,97.0
3927,Tata Steel Ltd,Materials,1339.10,87.0
2903,Power Grid Corporation of India Ltd,Utilities,231.10,82.0
2489,Muthoot Finance Ltd,Financials,1319.15,82.0
2670,Oil and Natural Gas Corporation Ltd,Energy,175.80,79.0
771,Coal India Ltd,Materials,189.45,78.0
454,Bajaj Auto Ltd,Consumer Discretionary,3714.60,76.0
1669,Indian Oil Corporation Ltd,Energy,128.10,74.0
1615,ITC Ltd,Consumer Staples,270.10,74.0


In [76]:
resultGrouped = resultDf[['name', 'price', 'marcap', 'sector', 'recommend']].groupby(['marcap', 'sector'])

In [78]:
resultGrouped.count()

name  price  recommend
marcap sector                                        
1                                 1      1          1
       Communication Services     4      4          4
       Consumer Discretionary     8      8          8
       Consumer Staples           6      6          6
       Financials                 3      3          3
       Industrials               11     11         11
       Information Technology     5      5          5
       Materials                 11     11         11
       Utilities                  3      3          3
2      Communication Services     1      1          1
       Consumer Discretionary     1      1          1
       Energy                     3      3          3
       Financials                 4      4          4
       Information Technology     1      1          1
       Materials                  4      4          4
       Utilities                  1      1          1
3      Communication Services     1      1          1
       Consumer Discretionary     1      1          1
       Consumer Staples           1      1          1
       Energy                     3      3          3
       Financials                 2      2          2
       Materials                  3      3          3
       Utilities                  1      1          1

In [77]:
resultGrouped.first()

name  \
marcap sector                                                           
1                                                            PCBL Ltd   
       Communication Services                                 MPS Ltd   
       Consumer Discretionary                      Nitin Spinners Ltd   
       Consumer Staples        Triveni Engineering and Industries Ltd   
       Financials                                Rail Vikas Nigam Ltd   
       Industrials                         Patels Airtemp (India) Ltd   
       Information Technology            Nucleus Software Exports Ltd   
       Materials                                             Moil Ltd   
       Utilities                                             SJVN Ltd   
2      Communication Services                      Sun Tv Network Ltd   
       Consumer Discretionary                       Hero MotoCorp Ltd   
       Energy                            Hindustan Petroleum Corp Ltd   
       Financials                       Power Finance Corporation Ltd   
       Information Technology  Oracle Financial Services Software Ltd   
       Materials                         Coromandel International Ltd   
       Utilities                                    Torrent Power Ltd   
3      Communication Services                        Indus Towers Ltd   
       Consumer Discretionary                          Bajaj Auto Ltd   
       Consumer Staples                                       ITC Ltd   
       Energy                        Bharat Petroleum Corporation Ltd   
       Financials                   Bajaj Holdings and Investment Ltd   
       Materials                                       Tata Steel Ltd   
       Utilities                  Power Grid Corporation of India Ltd   

                                 price  recommend  
marcap sector                                      
1                               117.75      100.0  
       Communication Services   621.20      100.0  
       Consumer Discretionary   268.05      100.0  
       Consumer Staples         339.05      100.0  
       Financials                34.35      100.0  
       Industrials              262.00      100.0  
       Information Technology   440.60      100.0  
       Materials                182.40      100.0  
       Utilities                 28.95      100.0  
2      Communication Services   505.65       67.0  
       Consumer Discretionary  2285.40       52.0  
       Energy                   283.35       82.0  
       Financials               119.00      100.0  
       Information Technology  3587.15      100.0  
       Materials                856.00      100.0  
       Utilities                570.50       22.0  
3      Communication Services   216.85       35.0  
       Consumer Discretionary  3714.60       76.0  
       Consumer Staples         270.10       74.0  
       Energy                   371.75       97.0  
       Financials              5204.40      100.0  
       Materials               1339.10       87.0  
       Utilities                231.10       82.0

In [93]:
resultDf.name.tolist()

['SJVN Ltd',
 'Moil Ltd',
 'Patels Airtemp (India) Ltd',
 'Nitin Spinners Ltd',
 'Garden Reach Shipbuilders & Engineers Ltd',
 'Triveni Engineering and Industries Ltd',
 'Rail Vikas Nigam Ltd',
 'JK Paper Ltd',
 'Nucleus Software Exports Ltd',
 'Graphite India Ltd',
 'Dalmia Bharat Sugar and Industries Ltd',
 'CARE Ratings Ltd',
 'Visaka Industries Ltd',
 'LG Balakrishnan & Bros Ltd',
 'MPS Ltd',
 'Sonata Software Ltd',
 'Avadh Sugar & Energy Ltd',
 'Coromandel International Ltd',
 'Lumax Industries Ltd',
 'Thanga Mayil Jewellery Ltd',
 'HEG Ltd',
 'Oracle Financial Services Software Ltd',
 'HIL Ltd',
 'Ncl Industries Ltd',
 'Bajaj Holdings and Investment Ltd',
 'Power Finance Corporation Ltd',
 'Ircon International Ltd',
 'Tamilnadu Petroproducts Ltd',
 'Dwarikesh Sugar Industries Ltd',
 'REC Limited',
 'Kirloskar Oil Engines Ltd',
 'PCBL Ltd',
 'Bharat Petroleum Corporation Ltd',
 'CESC Ltd',
 'Engineers India Ltd',
 'Tata Steel Ltd',
 'Manappuram Finance Ltd',
 'Welspun India Ltd',
